In [46]:
import torch 
from torch import utils
from torch import nn
import torch.nn.functional as F
from torch import optim
import pytorch_lightning as pl

import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = np.load("data/data.npz", allow_pickle=True)

train_data = data["train"]
test_data = data['test']

train_data.shape, test_data.shape

((5000, 2), (3000,))

In [19]:
class OPdataset(utils.data.Dataset):
    def __init__(self, data) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, i):
        input, target = self.data[i]
        return torch.tensor(input), torch.tensor(target)


In [28]:
from sklearn.utils import shuffle


dataset = OPdataset(train_data)
train_dataset, eval_dataset = utils.data.random_split(dataset, 
                                                    (int(len(dataset)*0.75), 
                                                    int(len(dataset)*0.25)))

train_dataloader = utils.data.DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
eval_dataloader = utils.data.DataLoader(eval_dataset, shuffle=True, batch_size=64, num_workers=4)

In [76]:
from turtle import forward


class ConvModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.vector_processor = nn.Sequential(
            nn.Linear(4,16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.ReLU()
        )
        self.points_computer = nn.Linear(300,2)
        self.loss_f = nn.CrossEntropyLoss()

    def forward(self, x):
        z = self.vector_processor(x)
        z = z.view(-1, 300)
        z = self.points_computer(z)
        z = F.softmax(z)

        return z

    def configure_optimizers(self):
        return optim.Adam(self.parameters())

    def training_step(self, batch):
        x, y = batch
        z = self.forward(x.float())
        loss = self.loss_f(z, y)
        self.log('training_loss', loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        z = self.forward(x.float())
        loss = self.loss_f(z, y)
        self.log('eval_loss', loss)

        return loss


In [77]:
model = MLPBaseline()
trainer = pl.Trainer()

trainer.fit(model, train_dataloader, eval_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name             | Type             | Params
------------------------------------------------------
0 | vector_processor | Sequential       | 969   
1 | points_computer  | Linear           | 602   
2 | loss_f           | CrossEntropyLoss | 0     
------------------------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/var/folders/v0/wj1l5yw5275_wypllbj2qhg40000gn/T/ipykernel_43523/1644298710.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = F.softmax(z)
/Users/macos/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/macos/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
